In [3]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google'

In [2]:
#install CATBOOST
pip install catboost

     |████████████████████████████████| 62.9MB 38kB/s 


In [320]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
import seaborn as seabornInstance 
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_absolute_error,mean_squared_error, r2_score
import lightgbm as lgb
import xgboost as xgb
# dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/Train_Data.csv')
# dataWithoutLabels = pd.read_csv(('/content/drive/My Drive/Colab Notebooks/ML/Test_Data.csv'))
# temp=pd.concat([dataset,dataWithoutLabels], ignore_index=True)
# print(dataset.tail)
# print(temp.columns.tolist())
#temp['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(temp['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
#print(temp['Yearly Income in addition to Salary (e.g. Rental Income)'].unique())

In [321]:
# col='Yearly Income in addition to Salary (e.g. Rental Income)'
# temp[col]=temp[col].map(lambda x: x.rstrip(' EUR'))
# print(temp[col].unique())
# a=temp[col].value_counts(dropna=False)
# print(a)
# X_train,X_test = temp.iloc[:1048574],temp.iloc[1048575:]
# print(X_train.head)
# rename_cols = {"Income in EUR":'Income'}
# train = train.rename(columns=rename_cols)
# for col in fill_col_dict.keys():
    # data[col] = data[col].fillna(fill_col_dict[col])
#X_train,X_test = data[features_col].iloc[:1048574],data[features_col].iloc[1048575:]

In [322]:

# to add noise on the data if dataframe has any NAN values
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))


#Function to apply Target encoding on categorial data it takes train and test column along with target column to give logical mean value to strings.

def target_encode(trn_series, tst_series,target):
    min_samples_leaf=1
    smoothing=1
    noise_level=0
    temp = pd.concat([trn_series, target], axis=1)
    
    # Compute target mean 
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    
    # Apply average function to all target data
    prior = target.mean()
    
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    
    # Apply averages to train and test series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index 
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)


In [323]:

def prepareData(dataset,dataWithoutLabels):
  #training data+test data
  
  dataset['Gender']=dataset['Gender'].replace(['unknown','0'], np.nan)  
  dataset['Gender']=dataset['Gender'].replace(['f'],'female')  
  dataset['Gender'] = dataset['Gender'].fillna(dataset['Gender'].mode()[0])
  dataset['Housing Situation']=dataset['Housing Situation'].replace(['nA',0,'0'], np.nan)
  dataset['Housing Situation'] = dataset['Housing Situation'].fillna(dataset['Housing Situation'].mode()[0]) 
    # dataset['Housing Situation']=dataset['Housing Situation'].replace(['Small House','Small Apartment'],'Small')
  # dataset['Housing Situation']=dataset['Housing Situation'].replace(['Large House','Large Apartment', 'Castle'],'Large')
  # dataset['Housing Situation']=dataset['Housing Situation'].replace(['Medium House','Medium Apartment'],'Medium')
  
  dataset['Work Experience in Current Job [years]']=pd.to_numeric(dataset['Work Experience in Current Job [years]'].replace(['#NUM!'],np.nan))
  dataset['Work Experience in Current Job [years]'] = dataset['Work Experience in Current Job [years]'].fillna(dataset['Work Experience in Current Job [years]'].median())
  #dataset['Satisfation with employer']=dataset['Satisfation with employer'].replace([np.nan],'Average')
  dataset['Satisfation with employer'] = dataset['Satisfation with employer'].fillna(dataset['Satisfation with employer'].mode()[0])
  dataset['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataset['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
  dataset['Hair Color']=dataset['Hair Color'].replace(['unknown','0'], np.nan)
  dataset['Year of Record'] = dataset['Year of Record'].fillna(dataset['Year of Record'].mean())  
  #dataset['Hair Color']=dataset['Hair Color'].replace([np.nan],'Brown')
  dataset['Hair Color'] = dataset['Hair Color'].fillna(dataset['Hair Color'].mode()[0])
  
  #print('fillna mean')
  #dataset=dataset.fillna(dataset.mean())
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['unknown','0'],np.nan)  
  dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace(['f'],'female')  
  #dataWithoutLabels['Gender']=dataWithoutLabels['Gender'].replace([np.nan],')
  dataWithoutLabels['Gender'] = dataWithoutLabels['Gender'].fillna(dataWithoutLabels['Gender'].mode()[0])                                                               
  
  dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['nA',0,'0'], np.nan)
  dataWithoutLabels['Housing Situation'] = dataWithoutLabels['Housing Situation'].fillna(dataWithoutLabels['Housing Situation'].mode()[0])
  # dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['Small House','Small Apartment'],'Small')
  # dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['Large House','Large Apartment', 'Castle'],'Large')
  # dataWithoutLabels['Housing Situation']=dataWithoutLabels['Housing Situation'].replace(['Medium House','Medium Apartment'],'Medium')

  dataWithoutLabels['Work Experience in Current Job [years]']=pd.to_numeric(dataWithoutLabels['Work Experience in Current Job [years]'].replace(['#NUM!'],np.nan))
  dataWithoutLabels['Work Experience in Current Job [years]'] = dataWithoutLabels['Work Experience in Current Job [years]'].fillna(dataWithoutLabels['Work Experience in Current Job [years]'].median())
  dataWithoutLabels['Satisfation with employer'] = dataWithoutLabels['Satisfation with employer'].fillna(dataWithoutLabels['Satisfation with employer'].mode()[0])  
  dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)']=pd.to_numeric(dataWithoutLabels['Yearly Income in addition to Salary (e.g. Rental Income)'].map(lambda x: x.rstrip(' EUR')))
  dataWithoutLabels['Hair Color']=dataWithoutLabels['Hair Color'].replace(['unknown','0'],np.nan)
  #dataset['Year of Record']=dataset['Year of Record'].replace([np.nan],'1942')
  dataset['Year of Record'] = dataset['Year of Record'].fillna(dataset['Year of Record'].mean())
  dataWithoutLabels['Hair Color'] = dataWithoutLabels['Hair Color'].fillna(dataWithoutLabels['Hair Color'].mode()[0])
  
  #dataWithoutLabels=dataWithoutLabels.fillna(method='ffill')
  #dataset=dataset.fillna(method='ffill')
    #encoding of categorical data

  #dataset['Housing Situation'],dataWithoutLabels['Housing Situation']=target_encode(dataset['Housing Situation'], dataWithoutLabels['Housing Situation'],dataset['Total Yearly Income [EUR]'])
  #dataset['Satisfation with employer'],dataWithoutLabels['Satisfation with employer']=target_encode(dataset['Satisfation with employer'], dataWithoutLabels['Satisfation with employer'],dataset['Total Yearly Income [EUR]'])
  #dataset['Gender'],dataWithoutLabels['Gender']=target_encode(dataset['Gender'], dataWithoutLabels['Gender'],dataset['Total Yearly Income [EUR]'])
  #dataset['Country'],dataWithoutLabels['Country']=target_encode(dataset['Country'], dataWithoutLabels['Country'],dataset['Total Yearly Income [EUR]'])
  #dataset['Profession'],dataWithoutLabels['Profession']=target_encode(dataset['Profession'], dataWithoutLabels['Profession'],dataset['Total Yearly Income [EUR]'])
  #dataset['University Degree'],dataWithoutLabels['University Degree']=target_encode(dataset['University Degree'], dataWithoutLabels['University Degree'],dataset['Total Yearly Income [EUR]'])
  #dataset['Hair Color'],dataWithoutLabels['Hair Color']=target_encode(dataset['Hair Color'], dataWithoutLabels['Hair Color'],dataset['Total Yearly Income [EUR]'])
  return dataset,dataWithoutLabels


In [324]:

  # plot the Values on a graph
def plotActualVsPredictedIncome(y_test, y_pred):
    df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
    df1 = df.head(25)
    df1.plot(kind='bar',figsize=(10,8))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
    plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.show()    

def trainModel(X_train, y_train, X_test, y_test):
   
    
    #Implementing linear regression (error rate 110000+)
    # trainingRegressor = LinearRegression()  
    # trainingRegressor.fit(X_train, y_train)
    # y_pred = trainingRegressor.predict(X_test)
    
    
    #Implementing RandomForest Regressor (error rate 80000+)
    #trainingRegressor = RandomForestRegressor(n_estimators=100)
    #trainingRegressor.fit(X_train,y_train)
    #y_pred = trainingRegressor.predict(X_test)
    
    #Implementing lightgbm    
    trainingRegressor = lgb.LGBMRegressor(num_leaves=300,
                                          learning_rate=0.05,
                                          n_estimators=10000)
    trainingRegressor.fit(X_train, y_train,
                          eval_set=[(X_test, y_test)],
                          eval_metric='l1',
                          early_stopping_rounds=5)


    
    
    print('Starting predicting...')
    # predict
    y_pred = trainingRegressor.predict(X_test, num_iteration=trainingRegressor.best_iteration_)
    
    
    #Implementing caatboost (error rate 60000+)
    #learning rate to get an optimal solution 
    
    # trainingRegressor = CatBoostRegressor(iterations=7000,learning_rate=0.02)
    # trainingRegressor.fit(X_train, y_train,use_best_model=True,verbose=True)
    # y_pred=trainingRegressor.predict(X_test)
    
    
    #evaluating the error
    df = pd.DataFrame({'Test': y_test, 'Prediction': y_pred})
    print("mean_absolute_error: %.2f"
      % mean_absolute_error(y_test, y_pred))
    
    #ploting the values on the graph
    #plotActualVsPredictedIncome(y_test, y_pred)

    return trainingRegressor
 

def runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels, encoder):
    print(featuresToConsider)
    print('dataset',dataWithoutLabels[featuresToConsider].dtypes)
    #get the values for X_test    
    
    X = dataWithoutLabels[featuresToConsider]
    X = encoder.transform(X)
    print('head',X)
    #df = pd.DataFrame(X)
    # scaler = StandardScaler()
    
    
    #scaler = sc.transform(df[['Year of Record', 'Crime Level in the City of Employement', 'Age', 'Size of City','Profession','Body Height [cm]', 'Yearly Income in addition to Salary (e.g. Rental Income)']])
    X = sc.fit_transform(X)
    #df['Year of Record'] = scaler[0]
    #df['Crime Level in the City of Employement'] = scaler[1]
    #df['Age'] = scaler[2]
    #df['Size of City'] = scaler[3]
    #df['Profession'] = scaler[4]
    #df['Body Height [cm]'] = scaler[5]
    #df['Yearly Income in addition to Salary (e.g. Rental Income)'] = scaler[6]
    #X = df
    #prediciting the values on the real test data
    predictedData = regressor.predict(X, num_iteration=regressor.best_iteration_)
    # predictedData_antilog = np.exp(predictedData)
    df = pd.DataFrame({'Predicted': predictedData})

    #write the data to csv
    # fileName = '/content/drive/My Drive/Colab Notebooks/ML/Sumission_file.csv'
    # df.to_csv(fileName, sep=',',index=False)
    
    dataset1 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/tcd-ml-1920-group-income-submission.csv')
    print("predicted",predictedData.shape)
    print("dataWithoutLabels",dataWithoutLabels.shape)
    print("dataset",dataset1.shape)
    dataset1['Instance'] = dataWithoutLabels['Instance']
    dataset1['Total Yearly Income [EUR]'] = predictedData
    dataset1.to_csv('/content/drive/My Drive/Colab Notebooks/ML/Sumission_file_new.csv', index=False)
    

    

#Reading data from CSV
dataset = pd.read_csv('/content/drive/My Drive/Colab Notebooks/ML/Train_Data.csv')
dataWithoutLabels = pd.read_csv(('/content/drive/My Drive/Colab Notebooks/ML/Test_Data.csv'))
#combined_data=pd.concat([dataset,dataWithoutLabels], ignore_index=True)

#preprocessing data on both the datasets



C:\Users\Spy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\Spy\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [325]:

dataset,dataWithoutLabels = prepareData(dataset,dataWithoutLabels)


In [327]:
featuresToConsider = ['Year of Record', 'Housing Situation', 'Crime Level in the City of Employement', 'Work Experience in Current Job [years]', 'Satisfation with employer', 'Gender', 'Age', 'Country', 'Size of City', 'Profession', 'Body Height [cm]', 'Yearly Income in addition to Salary (e.g. Rental Income)', 'University Degree']

#getting thte target data
y = dataset['Total Yearly Income [EUR]'].values

#Normalization
#dataset[featuresToConsider] = pd.DataFrame(scaler.fit_transform(dataset[featuresToConsider]), columns=featuresToConsider)
#dataWithoutLabels[featuresToConsider] = pd.DataFrame(scaler.fit_transform(dataWithoutLabels[featuresToConsider]), columns=featuresToConsider)
# print(dataset[featuresToConsider].dtypes)
# print(dataset[featuresToConsider].dtypes)
# print(dataset['Year of Record'].unique())

In [328]:

#Features Dataset
X = dataset[featuresToConsider]
import category_encoders as ce
target_encoder = ce.TargetEncoder(cols=['Housing Situation', 'Satisfation with employer', 'Gender', 'Country', 'Profession', 'University Degree'])
target_encoder.fit(X, y)
X = target_encoder.transform(X)


df = pd.DataFrame(X)
sc = preprocessing.MinMaxScaler()
X = sc.fit_transform(X)
#scaler = np.transpose(scaler)
#df['Year of Record'] = scaler[0]
#df['Crime Level in the City of Employement'] = scaler[1]
#df['Age'] = scaler[2]
#df['Size of City'] = scaler[3]
#df['Profession'] = scaler[4]
#df['Body Height [cm]'] = scaler[5]
#df['Yearly Income in addition to Salary (e.g. Rental Income)'] = scaler[6]
#X = df


#divide the training data into test and train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# y_train_log = np.log(y_train)

#get the trained model
regressor = trainModel(X_train, y_train, X_test, y_test)
# print(dataWithoutLabels.head)
# print(dataWithoutLabels.isnull().sum())
#run on the real test data using the. trained model
runTrainedModelOnActualData(regressor, featuresToConsider,dataWithoutLabels, target_encoder)

[1]	valid_0's l1: 69147.7	valid_0's l2: 1.14332e+10
Training until validation scores don't improve for 5 rounds
[2]	valid_0's l1: 65871.3	valid_0's l2: 1.04534e+10
[3]	valid_0's l1: 62767.3	valid_0's l2: 9.56654e+09
[4]	valid_0's l1: 59832.2	valid_0's l2: 8.76516e+09
[5]	valid_0's l1: 57049	valid_0's l2: 8.03877e+09
[6]	valid_0's l1: 54408.9	valid_0's l2: 7.37814e+09
[7]	valid_0's l1: 51909.4	valid_0's l2: 6.77754e+09
[8]	valid_0's l1: 49542.6	valid_0's l2: 6.23357e+09
[9]	valid_0's l1: 47298.6	valid_0's l2: 5.737e+09
[10]	valid_0's l1: 45183.7	valid_0's l2: 5.29061e+09
[11]	valid_0's l1: 43186.6	valid_0's l2: 4.88476e+09
[12]	valid_0's l1: 41296	valid_0's l2: 4.51499e+09
[13]	valid_0's l1: 39504.1	valid_0's l2: 4.17898e+09
[14]	valid_0's l1: 37819.6	valid_0's l2: 3.87528e+09
[15]	valid_0's l1: 36221.1	valid_0's l2: 3.59654e+09
[16]	valid_0's l1: 34716	valid_0's l2: 3.34575e+09
[17]	valid_0's l1: 33290.1	valid_0's l2: 3.11527e+09
[18]	valid_0's l1: 31953.7	valid_0's l2: 2.9075e+09
[19]

[156]	valid_0's l1: 9058.07	valid_0's l2: 4.97598e+08
[157]	valid_0's l1: 9056.02	valid_0's l2: 4.97477e+08
[158]	valid_0's l1: 9053.34	valid_0's l2: 4.97247e+08
[159]	valid_0's l1: 9046.14	valid_0's l2: 4.96714e+08
[160]	valid_0's l1: 9040.45	valid_0's l2: 4.96294e+08
[161]	valid_0's l1: 9037.97	valid_0's l2: 4.96067e+08
[162]	valid_0's l1: 9034.08	valid_0's l2: 4.95742e+08
[163]	valid_0's l1: 9029.8	valid_0's l2: 4.95336e+08
[164]	valid_0's l1: 9027.82	valid_0's l2: 4.9504e+08
[165]	valid_0's l1: 9018.34	valid_0's l2: 4.94309e+08
[166]	valid_0's l1: 9016.09	valid_0's l2: 4.93978e+08
[167]	valid_0's l1: 9014.07	valid_0's l2: 4.93735e+08
[168]	valid_0's l1: 9009	valid_0's l2: 4.93458e+08
[169]	valid_0's l1: 9000.27	valid_0's l2: 4.92839e+08
[170]	valid_0's l1: 8994.76	valid_0's l2: 4.92301e+08
[171]	valid_0's l1: 8992.65	valid_0's l2: 4.92217e+08
[172]	valid_0's l1: 8989.7	valid_0's l2: 4.91951e+08
[173]	valid_0's l1: 8987.93	valid_0's l2: 4.91822e+08
[174]	valid_0's l1: 8978.66	valid_

[310]	valid_0's l1: 8702.39	valid_0's l2: 4.69988e+08
[311]	valid_0's l1: 8701.6	valid_0's l2: 4.70002e+08
[312]	valid_0's l1: 8700.6	valid_0's l2: 4.6996e+08
[313]	valid_0's l1: 8699.55	valid_0's l2: 4.6985e+08
[314]	valid_0's l1: 8699.79	valid_0's l2: 4.69911e+08
[315]	valid_0's l1: 8699.07	valid_0's l2: 4.69893e+08
[316]	valid_0's l1: 8697.8	valid_0's l2: 4.69838e+08
[317]	valid_0's l1: 8697.25	valid_0's l2: 4.69859e+08
[318]	valid_0's l1: 8696.72	valid_0's l2: 4.69878e+08
[319]	valid_0's l1: 8695.99	valid_0's l2: 4.6975e+08
[320]	valid_0's l1: 8694.84	valid_0's l2: 4.69638e+08
[321]	valid_0's l1: 8693.35	valid_0's l2: 4.69593e+08
[322]	valid_0's l1: 8691.57	valid_0's l2: 4.69503e+08
[323]	valid_0's l1: 8690.97	valid_0's l2: 4.69513e+08
[324]	valid_0's l1: 8690.33	valid_0's l2: 4.69512e+08
[325]	valid_0's l1: 8689.58	valid_0's l2: 4.69415e+08
[326]	valid_0's l1: 8688.79	valid_0's l2: 4.6935e+08
[327]	valid_0's l1: 8687.72	valid_0's l2: 4.69246e+08
[328]	valid_0's l1: 8687.26	valid_0

[466]	valid_0's l1: 8572.21	valid_0's l2: 4.6361e+08
[467]	valid_0's l1: 8571.85	valid_0's l2: 4.63628e+08
[468]	valid_0's l1: 8571.48	valid_0's l2: 4.63628e+08
[469]	valid_0's l1: 8570.81	valid_0's l2: 4.6359e+08
[470]	valid_0's l1: 8570.48	valid_0's l2: 4.63601e+08
[471]	valid_0's l1: 8570.12	valid_0's l2: 4.63579e+08
[472]	valid_0's l1: 8569.55	valid_0's l2: 4.63568e+08
[473]	valid_0's l1: 8569.07	valid_0's l2: 4.6352e+08
[474]	valid_0's l1: 8568.18	valid_0's l2: 4.63475e+08
[475]	valid_0's l1: 8567.88	valid_0's l2: 4.63433e+08
[476]	valid_0's l1: 8567.44	valid_0's l2: 4.63386e+08
[477]	valid_0's l1: 8566.51	valid_0's l2: 4.63345e+08
[478]	valid_0's l1: 8565.91	valid_0's l2: 4.63344e+08
[479]	valid_0's l1: 8565.25	valid_0's l2: 4.6331e+08
[480]	valid_0's l1: 8565.19	valid_0's l2: 4.63302e+08
[481]	valid_0's l1: 8564.65	valid_0's l2: 4.63249e+08
[482]	valid_0's l1: 8564.3	valid_0's l2: 4.63234e+08
[483]	valid_0's l1: 8563.55	valid_0's l2: 4.63196e+08
[484]	valid_0's l1: 8563.16	valid

predicted (369438,)
dataWithoutLabels (369438, 17)
dataset (369438, 2)
